In [9]:
import sys
import pandas as pd
from pathlib import Path
import toml

sys.path.append('./abetam')

from abetam.data.canada import electricity_prices

config = toml.load('copper/scenarios/BAU_scenario/config.toml')

electricity_prices.head()

,Alberta,British Columbia,Canada,Manitoba,New Brunswick,Newfoundland and Labrador,Northwest Territories,Nova Scotia,Nunavut,Ontario,Prince Edward Island,Quebec,Saskatchewan,Yukon
REF_DATE,,,,,,,,,,,,,,
2000,8.402978,6.516774,10.105829,5.710638,7.342644,NaN,NaN,9.751083,NaN,7.211406,9.030832,4.764065,10.126978,NaN
2001,8.419454,6.692903,10.433143,6.073404,7.240460,NaN,NaN,9.561188,NaN,7.735599,9.388489,4.699748,11.074268,NaN
2002,8.238213,6.774194,10.228571,5.851064,7.298851,NaN,NaN,9.494725,NaN,7.488479,8.941418,4.594084,11.277259,NaN
2003,9.819950,7.221290,11.036629,6.137766,7.773276,NaN,NaN,10.235314,NaN,8.117512,9.942857,4.832977,11.920062,NaN
2004,10.198908,7.736129,11.783314,6.576596,8.349885,NaN,NaN,11.042365,NaN,8.641705,10.801233,5.218880,12.641807,NaN


In [10]:
def add_province(df):
    if "province" in df.columns:
        return df
    else:
        df.reset_index(inplace=True)
        df["province"] = df["node"].str.split(".", expand=True)[0]
        return df


copper_result_dir = "copper/results/LastRun"
duals = pd.read_csv(f"{copper_result_dir}/duals.csv").set_index(
    ["year", "hour", "node"]
)

duals = add_province(duals)

run_day_key = "run_days_test"
n_days = len(config["Simulation_Settings"][run_day_key])
print(
    f"n_days: {n_days}, using {run_day_key}: {config['Simulation_Settings'][run_day_key]}"
)

mean_electricity_price = (
    duals.groupby(["year", "province"])["dual_price"].mean() * -n_days / 365
)
mean_electricity_price /= 10 # $/MWh -> c/kWh
mean_electricity_price

n_days: 1, using run_days_test: [2]


year  province
2025  Ontario     13.333921
2030  Ontario     11.758158
2035  Ontario      9.867423
2040  Ontario      8.102771
2045  Ontario      6.640926
2050  Ontario      0.000337
Name: dual_price, dtype: float64

In [11]:
el_price_copper = mean_electricity_price.to_frame().reset_index().pivot(index="year", columns="province", values="dual_price")
el_price_copper

province,Ontario
year,
2025,13.333921
2030,11.758158
2035,9.867423
2040,8.102771
2045,6.640926
2050,0.000337


In [12]:
el_price_path = "abetam/data/canada/ca_electricity_prices.csv"
el_prices_df = pd.read_csv(el_price_path).set_index("REF_DATE")
el_prices_df.head()

,Alberta,British Columbia,Canada,Manitoba,New Brunswick,Newfoundland and Labrador,Northwest Territories,Nova Scotia,Nunavut,Ontario,Prince Edward Island,Quebec,Saskatchewan,Yukon
REF_DATE,,,,,,,,,,,,,,
2000,8.402978,6.516774,10.105829,5.710638,7.342644,NaN,NaN,9.751083,NaN,7.211406,9.030832,4.764065,10.126978,NaN
2001,8.419454,6.692903,10.433143,6.073404,7.240460,NaN,NaN,9.561188,NaN,7.735599,9.388489,4.699748,11.074268,NaN
2002,8.238213,6.774194,10.228571,5.851064,7.298851,NaN,NaN,9.494725,NaN,7.488479,8.941418,4.594084,11.277259,NaN
2003,9.819950,7.221290,11.036629,6.137766,7.773276,NaN,NaN,10.235314,NaN,8.117512,9.942857,4.832977,11.920062,NaN
2004,10.198908,7.736129,11.783314,6.576596,8.349885,NaN,NaN,11.042365,NaN,8.641705,10.801233,5.218880,12.641807,NaN


In [13]:
for year in el_price_copper.index:
    for prov in el_price_copper.columns:
        el_prices_df.at[year, prov] = el_price_copper.loc[year, prov]
el_prices_df.to_csv(el_price_path)

In [14]:
results_dir = Path("abetam/results/master/faf4d77a60266d460a2791eb2e28a4db")


In [16]:
r_dir_name_base = results_dir.name
r_dir_name_rerun = r_dir_name_base + "_{i}"
i = 0
while results_dir.with_name(r_dir_name_rerun.format(i=i)).exists():
    i += 1
results_dir_rerun = results_dir.with_name(r_dir_name_rerun.format(i=i))
results_dir_rerun.mkdir(exist_ok=True)
results_dir_rerun


WindowsPath('abetam/results/master/faf4d77a60266d460a2791eb2e28a4db_4')

In [1]:
import sys
sys.path.append("abetam")

from abetam.batch import BatchResult

from abetam.components.probability import beta_with_mode_at
best_tech_modes = {
    "Electric furnace": 0.788965,
    "Gas furnace": 0.388231,
    "Heat pump": 0.346424,
    "Oil furnace": 0.460226,
    "Wood or wood pellets furnace": 0.516056,
}
batch_parameters = {
        "N": [50],
        "province": ["Ontario"],
        "random_seed": list(range(42, 44)),
        "tech_attitude_dist_func": beta_with_mode_at,
        "tech_attitude_dist_params": [best_tech_modes],
        "start_year": 2020,
        "price_weight_mode": 0.875,
    }

BatchResult.get_results_dir(batch_parameters, force_rerun=True)


WindowsPath('c:/src/canadopt/abetam/results/master/85a295abbfc554f4867d13af9d78ac31_1')

In [2]:
result = BatchResult.from_parameters(batch_parameters, force_rerun=True)

results_dir=WindowsPath('c:/src/canadopt/abetam/results/master/85a295abbfc554f4867d13af9d78ac31_1') does not exist. Running model.


100%|██████████| 2/2 [00:07<00:00,  3.52s/it]


In [3]:
result.save()

WindowsPath('c:/src/canadopt/abetam/results/master/85a295abbfc554f4867d13af9d78ac31_1')